# TNT Cates: Stress Relaxation
> **Handbook:** This notebook demonstrates the TNT Cates model. For complete mathematical derivations and theoretical background, see [TNT Cates Documentation](../../docs/source/models/tnt/tnt_cates.rst).


**Estimated Time:** 3-5 minutes

## Protocol: Stress Relaxation in Living Polymers

**Stress relaxation** in living polymers exhibits **bi-exponential decay** — a hallmark of the Cates model's two-timescale physics.

### Physical Context for Living Polymers

After a step strain is applied, the stress decays through two mechanisms:

1. **Fast mode** ($\tau_{break}$): Chain scission quickly releases some stress by reducing average chain length
2. **Slow mode** ($\tau_{rep}$): Remaining chains relax via reptation (diffusion along their contour)

The relaxation modulus takes the form:

$$
G(t) = G_1 \exp(-t/\tau_1) + G_2 \exp(-t/\tau_2)
$$

where $\tau_1, \tau_2$ are related to $\tau_{break}$ and $\tau_{rep}$, with the **dominant observable timescale**:

$$
\tau_d = \sqrt{\tau_{rep} \cdot \tau_{break}}
$$

This bi-exponential form contrasts with:
- **TNT Single-Mode**: Single exponential $G(t) = G \exp(-t/\tau_b)$
- **Sticky Rouse**: Multi-exponential with hierarchical mode spectrum

> **Cates Relaxation Physics**  
> For bi-exponential analysis and timescale extraction, see:  
> [../../docs/source/models/tnt/tnt_cates.rst](../../docs/source/models/tnt/tnt_cates.rst) — Section on "Stress Relaxation"

---

## Learning Objectives

1. Understand bi-exponential relaxation in living polymers
2. Fit TNT Cates model to relaxation modulus data
3. Extract $\tau_{break}$ (fast) and $\tau_{rep}$ (slow) from decay
4. Verify $\tau_d = \sqrt{\tau_{rep} \tau_{break}}$ consistency with SAOS
5. Compare bi-exponential vs. single-exponential (TNT Single-Mode)
6. Perform Bayesian inference for parameter uncertainty

## Prerequisites

- TNT Cates fundamentals (Notebook 07)
- Understanding of stress relaxation (TNT Single-Mode NB03)
- Familiarity with multi-exponential fitting

## Runtime Estimate

- NLSQ fitting: ~3-8 seconds
- Bayesian inference (demo): ~45-90 seconds
- Total: ~3-5 minutes

## Setup

In [ ]:
import os
import sys
import time

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

from rheojax.core.jax_config import safe_import_jax

jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64

verify_float64()

from rheojax.models.tnt import TNTCates

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    compute_cates_tau_d,
    compute_fit_quality,
    get_tnt_cates_param_names,
    load_laponite_relaxation,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
)
from utils.plotting_utils import (
    display_arviz_diagnostics,
    plot_nlsq_fit,
    plot_posterior_predictive,
)

param_names = get_tnt_cates_param_names()
print(f"TNTCates parameters: {param_names}")

# Residual analysis
residuals = stress - model.predict(gamma_dot, test_mode="flow_curve")
print(f"\nResidual Statistics:")
print(f"  Mean residual = {np.mean(residuals):.4e}")
print(f"  Std residual = {np.std(residuals):.4e}")
print(f"  Max absolute residual = {np.max(np.abs(residuals)):.4e}")


### Bayesian Convergence Diagnostics

When running full Bayesian inference (FAST_MODE=0), monitor these diagnostic metrics to ensure MCMC convergence:

| Metric | Acceptable Range | Interpretation |
|--------|------------------|----------------|
| **R-hat** | < 1.01 | Measures chain convergence; values near 1.0 indicate chains mixed well |
| **ESS (Effective Sample Size)** | > 400 | Number of independent samples; higher is better |
| **Divergences** | < 1% of samples | Indicates numerical instability; should be near zero |
| **BFMI (Bayesian Fraction of Missing Information)** | > 0.3 | Low values suggest reparameterization needed |

**Troubleshooting poor diagnostics:**
- High R-hat (>1.01): Increase `num_warmup` or `num_chains`
- Low ESS (<400): Increase `num_samples` or check for strong correlations
- Many divergences: Increase `target_accept` (default 0.8) or use NLSQ warm-start


## Theory: Cates Model for Stress Relaxation

The Cates model predicts stress relaxation following step strain:

**Relaxation modulus:**
In the fast-breaking limit ($\tau_{\text{break}} \ll \tau_{\text{rep}}$):
$$G(t) \approx G_0 \exp(-t/\tau_d) + G_\infty$$

where $\tau_d = \sqrt{\tau_{\text{rep}} \cdot \tau_{\text{break}}}$ is the effective relaxation time.

**Comparison with Maxwell:**
- TNTSingleMode: $G(t) = G_0 \exp(-t/\tau)$ (purely reptation)
- TNTCates: Same exponential form but $\tau_d$ combines reptation and breaking

**Physical interpretation:**
- Faster breaking → shorter $\tau_d$ → faster relaxation
- $\zeta = \tau_{\text{break}}/\tau_{\text{rep}} \ll 1$ for fast-breaking

## Load Stress Relaxation Data

In [ ]:
time_data, G_t = load_laponite_relaxation(aging_time=1800)

print(f"Data points: {len(time_data)}")
print(f"Time range: {time_data.min():.2e} to {time_data.max():.2e} s")
print(f"Modulus range: {G_t.min():.2e} to {G_t.max():.2e} Pa")

fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(time_data, G_t, 'o', label='Experimental data', markersize=5)
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Relaxation Modulus G(t) (Pa)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_title('Stress Relaxation Data', fontsize=14)
plt.close("all")
plt.close('all')

## NLSQ Fitting

In [ ]:
model = TNTCates()

# Relaxation mode requires pre-shear rate (rate used to achieve initial strain)
# For step strain relaxation, this represents the initial straining condition
gamma_dot = 1.0  # s^-1, typical value for step strain experiments

start_time = time.time()
model.fit(time_data, G_t, test_mode='relaxation', gamma_dot=gamma_dot, method='scipy')
fit_time = time.time() - start_time

print(f"\nNLSQ Optimization completed in {fit_time:.2f} seconds")

# Extract fitted parameters
nlsq_params = {name: model.parameters.get_value(name) for name in param_names}
print("\nNLSQ Parameters:")
for name, value in nlsq_params.items():
    print(f"  {name}: {value:.4e}")

# Compute fit quality
G_pred_fit = model.predict(time_data, test_mode='relaxation', gamma_dot=gamma_dot)
quality = compute_fit_quality(G_t, G_pred_fit)
print(f"\nFit Quality: R² = {quality['R2']:.6f}, RMSE = {quality['RMSE']:.4e}")

## Visualize NLSQ Fit

In [ ]:
# Compute metrics for plot title
metrics = compute_fit_quality(G_t, model.predict(time_data, test_mode='relaxation', gamma_dot=gamma_dot))

# Plot NLSQ fit with uncertainty band
fig, ax = plot_nlsq_fit(
    time_data, G_t, model, test_mode="relaxation",
    param_names=param_names, log_scale=True,
    xlabel='Time (s)',
    ylabel=r'Relaxation modulus $G(t)$ (Pa)',
    title=f'NLSQ Fit (R² = {metrics["R2"]:.4f})',
    gamma_dot=gamma_dot
)
plt.close("all")

## Physical Analysis: Extract tau_d from Exponential Fit

In [ ]:
tau_d_cates = compute_cates_tau_d(nlsq_params['tau_rep'], nlsq_params['tau_break'])
zeta = nlsq_params['tau_break'] / nlsq_params['tau_rep']

def exponential_decay(t, G0, tau, Ginf):
    return G0 * np.exp(-t / tau) + Ginf

popt, _ = curve_fit(
    exponential_decay,
    np.array(time_data),
    np.array(G_t),
    p0=[nlsq_params['G_0'], tau_d_cates, nlsq_params['eta_s']*1e-6],
    maxfev=5000
)
G0_exp, tau_d_exp, Ginf_exp = popt

print(f"\nPhysical Analysis:")
print(f"  Reptation time (tau_rep): {nlsq_params['tau_rep']:.4e} s")
print(f"  Breaking time (tau_break): {nlsq_params['tau_break']:.4e} s")
print(f"  Effective relaxation time (tau_d from Cates): {tau_d_cates:.4e} s")
print(f"  Effective relaxation time (tau_d from exp fit): {tau_d_exp:.4e} s")
print(f"  Agreement: {abs(tau_d_cates - tau_d_exp)/tau_d_cates * 100:.1f}% difference")
print(f"  Fast-breaking parameter (zeta): {zeta:.4f}")

if zeta < 0.1:
    print(f"\n  → Fast-breaking limit: Exponential decay expected")
else:
    print(f"\n  → Not in fast-breaking limit: May show deviations from exponential")

## Compare with Single-Mode Maxwell

In [ ]:
# Generate fine grid for smooth predictions
time_pred = np.linspace(time_data.min(), time_data.max(), 200)
G_pred = model.predict(time_pred, test_mode='relaxation', gamma_dot=gamma_dot)

from rheojax.models.tnt import TNTSingleMode

maxwell_model = TNTSingleMode()
# Map TNTCates parameters to TNTSingleMode:
# G_0 -> G, tau_d (effective) -> tau_b, eta_s -> eta_s
maxwell_model.parameters.set_value('G', nlsq_params['G_0'])
maxwell_model.parameters.set_value('tau_b', tau_d_cates)
maxwell_model.parameters.set_value('eta_s', nlsq_params['eta_s'])

maxwell_pred = maxwell_model.predict(time_pred, test_mode='relaxation', gamma_dot=gamma_dot)

fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(time_data, G_t, 'o', label='Data', markersize=5, zorder=3)
ax.loglog(time_pred, G_pred, '-', linewidth=2, label='TNTCates', zorder=2)
ax.loglog(time_pred, maxwell_pred, '--', linewidth=2, label=f'Maxwell (τ={tau_d_cates:.2e}s)', zorder=1)
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('G(t) (Pa)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_title('Cates vs Maxwell Comparison', fontsize=14)
plt.close("all")
plt.close('all')

## Bayesian Inference with NUTS

In [ ]:
# FAST_MODE: Use reduced MCMC for quick validation
# FAST_MODE controls Bayesian inference (env var FAST_MODE, default=1)
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

# Configuration
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

if FAST_MODE:
    print("FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)")
    print("To run Bayesian analysis, run with FAST_MODE=0")
    # Create a placeholder result with current NLSQ parameters
    class BayesianResult:
        def __init__(self, model, param_names):
            self.posterior_samples = {name: np.array([model.parameters.get_value(name)] * NUM_SAMPLES) for name in param_names}
    bayesian_result = BayesianResult(model, param_names)
    bayes_time = 0.0
else:
    print(f"Running NUTS with {NUM_CHAINS} chain(s)...")
    print(f"Warmup: {NUM_WARMUP} samples, Sampling: {NUM_SAMPLES} samples")
    
    start_time = time.time()
    bayesian_result = model.fit_bayesian(
        time_data, G_t,
        test_mode='relaxation',
        gamma_dot=gamma_dot,
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        seed=42
    )
    bayes_time = time.time() - start_time
    
    print(f"\nBayesian inference completed in {bayes_time:.1f} seconds")


## Convergence Diagnostics

In [ ]:
posterior = bayesian_result.posterior_samples

bayesian_params = {name: float(jnp.mean(posterior[name])) for name in param_names}
param_std = {name: float(jnp.std(posterior[name])) for name in param_names}

print("\nPosterior Statistics:")
for name in param_names:
    print(f"  {name}: {bayesian_params[name]:.4e} ± {param_std[name]:.4e}")

# Compare NLSQ vs Bayesian using the utility function
print_parameter_comparison(model, posterior, param_names)

## ArviZ Diagnostics

In [ ]:
# ArviZ diagnostics (trace, pair, forest, energy, autocorrelation, rank)
if not FAST_MODE and hasattr(bayesian_result, 'to_inference_data'):
    display_arviz_diagnostics(bayesian_result, param_names, fast_mode=FAST_MODE)
else:
    print("FAST_MODE: Skipping ArviZ diagnostics")

## Posterior Predictive Distribution

In [ ]:
# Posterior predictive check
if not FAST_MODE and hasattr(bayesian_result, 'posterior_samples'):
    fig, ax = plot_posterior_predictive(
        time_data,
        G_t,
        model, bayesian_result, test_mode="relaxation",
        param_names=param_names, log_scale=True,
        xlabel=r'Time (s)',
        ylabel=r'Relaxation modulus $G(t)$ (Pa)', gamma_dot=gamma_dot
    )
    plt.close("all")
else:
    print("FAST_MODE: Skipping posterior predictive")

## Physical Interpretation from Posterior

In [ ]:
tau_d_posterior = np.sqrt(posterior['tau_rep'] * posterior['tau_break'])
zeta_posterior = posterior['tau_break'] / posterior['tau_rep']

print(f"\nPhysical quantities from posterior:")
print(f"  tau_d: {np.mean(tau_d_posterior):.4e} ± {np.std(tau_d_posterior):.4e} s")
print(f"  zeta (tau_break/tau_rep): {np.mean(zeta_posterior):.4f} ± {np.std(zeta_posterior):.4f}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.hist(tau_d_posterior, bins=30, alpha=0.7, edgecolor='black')
ax1.axvline(np.mean(tau_d_posterior), color='r', linestyle='--', label='Mean')
ax1.set_xlabel(r'$\tau_d$ (s)', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.legend()
ax1.set_title('Effective Relaxation Time Distribution', fontsize=12)

ax2.hist(zeta_posterior, bins=30, alpha=0.7, edgecolor='black')
ax2.axvline(np.mean(zeta_posterior), color='r', linestyle='--', label='Mean')
ax2.axvline(0.1, color='g', linestyle=':', label='Fast-breaking limit')
ax2.set_xlabel(r'$\zeta = \tau_{break}/\tau_{rep}$', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.legend()
ax2.set_title('Fast-Breaking Parameter Distribution', fontsize=12)

plt.tight_layout()
plt.close("all")
plt.close('all')

## Save Results

In [ ]:
save_tnt_results(model, bayesian_result, "cates", "relaxation", param_names)

## Key Takeaways

1. **Exponential relaxation** in Cates model arises from effective time scale $\tau_d = \sqrt{\tau_{\text{rep}} \cdot \tau_{\text{break}}}$
2. **Fast-breaking limit** gives Maxwell-like exponential decay: $G(t) \sim \exp(-t/\tau_d)$
3. **Direct extraction** of $\tau_d$ from exponential fit agrees with Cates parameters
4. **Physical equivalence** with single-mode Maxwell but with different microscopic origin (breaking vs pure reptation)
5. **Bayesian inference** provides uncertainty quantification for time scales from relaxation data

**Next steps:** Compare $\tau_d$ across protocols (flow curve, startup, SAOS) to verify consistency.

## Next Steps

- **Notebook 10**: Continue exploring this model family
- **Advanced Models**: Compare with other TNT variants (Notebooks 07-30)


## Further Reading

### TNT Documentation

- **[TNT Model Family Overview](../../docs/source/models/tnt/index.rst)**: Complete guide to all 5 TNT models
- **[TNT Protocols Reference](../../docs/source/models/tnt/tnt_protocols.rst)**: Mathematical framework for all protocols
- **[TNT Knowledge Extraction](../../docs/source/models/tnt/tnt_knowledge_extraction.rst)**: Guide for interpreting fitted parameters

### Related Notebooks

Explore other protocols in this model family and compare with advanced TNT models.


### Key References

1. **Tanaka, F., & Edwards, S. F.** (1992). Viscoelastic properties of physically crosslinked networks. 1. Transient network theory. *Macromolecules*, 25(5), 1516-1523. [DOI: 10.1021/ma00031a024](https://doi.org/10.1021/ma00031a024)
   - **Original TNT framework**: Conformation tensor dynamics for reversible networks

2. **Green, M. S., & Tobolsky, A. V.** (1946). A new approach to the theory of relaxing polymeric media. *Journal of Chemical Physics*, 14(2), 80-92. [DOI: 10.1063/1.1724109](https://doi.org/10.1063/1.1724109)
   - **Transient network foundation**: Network strand creation and breakage kinetics

3. **Yamamoto, M.** (1956). The visco-elastic properties of network structure I. General formalism. *Journal of the Physical Society of Japan*, 11(4), 413-421. [DOI: 10.1143/JPSJ.11.413](https://doi.org/10.1143/JPSJ.11.413)
   - **Network viscoelasticity theory**: Mathematical formulation of temporary networks

4. **Bell, G. I.** (1978). Models for the specific adhesion of cells to cells. *Science*, 200(4342), 618-627. [DOI: 10.1126/science.347575](https://doi.org/10.1126/science.347575)
   - **Bell breakage model**: Stress-dependent bond dissociation kinetics

5. **Sprakel, J., Spruijt, E., Cohen Stuart, M. A., van der Gucht, J., & Besseling, N. A. M.** (2008). Universal route to a state of pure shear flow. *Physical Review Letters*, 101(24), 248304. [DOI: 10.1103/PhysRevLett.101.248304](https://doi.org/10.1103/PhysRevLett.101.248304)
   - **TNT experimental validation**: Flow curve measurements and rheological signatures
